<a href="https://colab.research.google.com/github/VishanOberoi/LLMs/blob/main/Chainlit_%2B_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_experimental openai chainlit ctransformers langchain_community pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.6/171.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 10.3 MB/s

In [ ]:
%%writefile app.py

import chainlit as cl
from langchain_community.llms import CTransformers
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools

config = {'gpu_layers' : 50,
          'max_new_tokens' : 512,
          'temperature' : 0.3,
          'top_k' : 40,
          'top_p' : 0.6,
        }


llm = CTransformers(
    model = "vishanoberoi/Llama-2-7b-chat-hf-finedtuned-to-GGUF",
    model_file="finetuned.gguf",
    model_type="llama",
    callbacks=[StreamingStdOutCallbackHandler()],
    config = config
    )


template = """

As a chatbot for the Company titled 'Maitri Lab Grown Diamonds', our primary focus is to provide very structured concise, accurate and helpful information about the company Maitri Lab Grown diamonds and the diamond industry. Engage in discussions related to diamonds, including sourcing, types, and care, as well as information specific to Maitri Diamonds. When asked about unrelated topics, politely redirect the conversation to the company, your areas of expertise or inform the user that the topic falls outside your scope. Maintain a professional and neutral tone, respecting all viewpoints and ensuring accuracy in your responses.
Maitri is one word.
Question: {question}

Answer:"""

prompt = PromptTemplate.from_template(template)

memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=2, return_messages=True
)

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory
)

@cl.on_chat_start
def main():
    prompt = PromptTemplate(template = template, input_variables = ['question'])
    llm_chain = LLMChain(prompt = prompt, llm = conversation, verbose = False)

    # Storing the chain in the user session
    cl.user_session.set("llm_chain", llm_chain)

@cl.on_message
async def main1(message: str):

    llm_chain = cl.user_session.get('llm_chain')

    # Method 1
    # res = await llm_chain.acall(message , callbacks= [cl.AsyncLangchainCallbackHandler()])
    # await cl.Message(content = res['text']).send()

    # Method 2
    # cb = cl.LangchainCallbackHandler(stream_final_answer=True)
    # await cl.make_async(llm_chain.run)(message, callbacks=[cb])

    # Method 3
    # cb = cl.AsyncLangchainCallbackHandler(
        # stream_final_answer=True
    # )
    # res = await llm_chain.ainvoke(message, callbacks=[cb])
    # await cl.Message(content=res['text']).send()

    # Method 4
    cb = cl.LangchainCallbackHandler(stream_final_answer=True)
    await cl.make_async(llm_chain.invoke)(message, callbacks=[cb])






Overwriting app.py


In [29]:
import logging

# Setup logging
logging.basicConfig(filename='app_debug.log', level=logging.DEBUG, format='%(asctime)s %(levelname)s:%(message)s')

In [58]:
%%writefile app1.py

import chainlit as cl
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import logging



# Configuration for the language model
config = {
    'gpu_layers': 100,
    'max_new_tokens': 500,
    'context_length': 4000,
    'temperature': 0.2,
    'top_k': 40,
    'top_p': 0.6
}

llm = CTransformers(
    model = "vishanoberoi/Llama-2-7b-chat-hf-finedtuned-to-GGUF",
    model_file="finetuned.gguf",
    model_type="llama",
    callbacks=[StreamingStdOutCallbackHandler()],
    config = config
    )


# Define the prompt template
template = '''
[INST]
<<SYS>>
As a chatbot for the Company "Maitri Lab Grown Diamonds," your primary focus is to provide concise, accurate, and helpful information about the company and the diamond industry. Engage in discussions related to diamonds, including sourcing, types, and care, as well as information specific to Maitri Diamonds. When asked about unrelated topics, politely redirect the conversation to the company, your areas of expertise, or inform the user that the topic falls outside your scope. Maintain a professional and neutral tone, respecting all viewpoints and ensuring accuracy in your responses.
Respond to the following question as a chatbot for the company named "MAITRI LAB GROWN DIAMONDS."
<</SYS>>

{question}[/INST]'''

# Define the chat start function
@cl.on_chat_start
def on_chat_start():
    prompt = PromptTemplate(template=template, input_variables=["question"])
    llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=False)

    cl.user_session.set("llm_chain", llm_chain)


# Define the message function
@cl.on_message
async def on_message(message: cl.Message):
    llm_chain = cl.user_session.get("llm_chain")


    cb = cl.AsyncLangchainCallbackHandler(
        stream_final_answer=True
    )
    res = await llm_chain.acall(message.content, callbacks = [cb])


    await cl.Message(content=res['text']).send()

# Ensure you choose an available port or manage the port conflict.




Overwriting app1.py


In [62]:
!ngrok config add-authtoken '2ciYBEFmJ18NhKfa6HIWbMTJPj6_6fnKCmP1dBwEdbA3WZ4xD'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [63]:
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

Public URL: https://d816-34-83-176-159.ngrok-free.app


In [64]:
!chainlit run app1.py

/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py:548: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.llms import CTransformers`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
Fetching 1 files: 100% 1/1 [00:00<00:00, 3264.05it/s]
Fetching 1 files: 100% 1/1 [00:00<00:00, 3584.88it/s]
2024-02-22 13:31:59 - Your app is available at http://localhost:8000
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `acall` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use ainvoke instead.
  warn_deprecated(
  Hello! Welcome to Maitri Lab Grown Diamonds! I'm here to help you with any questions or concerns you may have about our company, the diamond industry, or our unique lab-g

In [ ]:
ngrok.kill()